In [ ]:
# ① モジュールのインストール
# 必要なモジュールをインストールする。
!pip install lxml cssselect

In [ ]:
import os
import re
import codecs
import requests
import urllib
from urllib.request import urlopen
import lxml.html
import zipfile
 
import numpy as np
import pandas as pd
 
from google.colab import auth

# Colaboratory上でのルートパス
ROOT_DIR = '.' + os.path.sep + 'aozora' + os.path.sep

# ダウンロードするzipファイルを格納するディレクトリ
ZIP_DIR = ROOT_DIR + 'zip' + os.path.sep

# 解凍したテキストファイルを格納するディレクトリ(文字コードはsjis)
TEXT_SJIS_DIR = ROOT_DIR + 'text_sjis' + os.path.sep

# sjisのテキストファイルをutf8に変換したファイルを格納するディレクトリ(文字コードはutf8)
TEXT_UTF8_DIR = ROOT_DIR + 'text_utf8' + os.path.sep

# テキストファイルを格納するGCSのパス
# (「バケット名」は使用するバケット名に変更する)
GS_DIR = 'gs://バケット名/aozora_dataset/'

# 学習データリストのcsvファイル名
DATA_LIST_FILENAME = 'data_list.csv'

# 青空文庫のURL 
base_url = 'http://www.aozora.gr.jp/'

# ダウンロード対象ファイルリストを作成する。
#  author : AutoMLでラベルとなる作家名
#  url : 作家ごとの作品一覧URL
#  download_num : ダウンロードする作品数
#  exclusion_list : ダウンロード対象から除外する作品名
author_dict = {
    'author' : [
                'NatsumeSoseki', 
                'DazaiOsamu', 
                'MiyazawaKenji',
                'AkutagawaRyunosuke',
                'YumenoKyusaku',
                'FukuzawaYukichi'
                ], 
    'url' : [
             'http://www.aozora.gr.jp/index_pages/person148.html#sakuhin_list_1',
             'https://www.aozora.gr.jp/index_pages/person35.html#sakuhin_list_1',
             'https://www.aozora.gr.jp/index_pages/person81.html#sakuhin_list_1',
             'https://www.aozora.gr.jp/index_pages/person879.html#sakuhin_list_1',
             'https://www.aozora.gr.jp/index_pages/person96.html#sakuhin_list_1',
             'https://www.aozora.gr.jp/index_pages/person296.html#sakuhin_list_1'
            ],
    'download_num' : [
                      12, 
                      23, 
                      27,
                      20,
                      8,
                      35
                      ],
    'exclusion_list' : [
        ['こころ', '永日小品'],
        ['走れメロス'],
        ['銀河鉄道の夜', '〔青びかる天弧のはてに〕', '青柳教諭を送る', '〔あくたうかべる朝の水〕'],
        ['羅生門'],
        ['ドグラ・マグラ'],
        ['学問のすすめ']
    ]
}
author_df = pd.DataFrame(author_dict)


# ② 青空文庫のzipファイルをダウンロード
# ダウンロード対象ファイルリストで定義した作品のテキストデータをダウンロードする。
def download_zip(author_df):
    for index, row in author_df.iterrows():
        zip_dict = {}
 
        # 作家ごとの作品リストHTMLを取得
        url = row['url']
        resp_root = requests.get(url)
        html_root = lxml.html.fromstring(resp_root.content)
        html_root.make_links_absolute(resp_root.url)
 
        print('author : {}'.format(row['author']))
        print('url : {}'.format(url))
 
        download_count = 0
        for a in html_root.cssselect('a'):
            # 作家ごとに定義した作品数をダウンロードしたら終了
            if -1 < row['download_num'] and row['download_num'] <= download_count:
                break
 
            # 除外する作品はスキップ
            if a.text in row['exclusion_list']:
                print('    {} is excluded'.format(a.text))
                continue
 
            link = a.get('href')
            if link is not None and 'cards' in link:
                # 図書カードページのHTMLを取得
                resp_card = requests.get(link)
                html_card = lxml.html.fromstring(resp_card.content)
                html_card.make_links_absolute(resp_card.url)
                link_zip_list = html_card.xpath('//a[contains(@href, ".zip")]')
                if 0 < len(link_zip_list):
                    link_zip = link_zip_list[0]
                    zip_filename = link_zip.text.split('.')[-2]
                    zip_link = link_zip.get('href')
                    print('[{}],{},{}'. format(a.text, zip_filename, zip_link))
 
                    # ダウンロード対象としてzipファイルのURLを取得
                    zip_dict[zip_filename] = zip_link
                    download_count += 1
 
        download_path = ZIP_DIR + row['author'] + os.path.sep
        if not os.path.exists(download_path):
            os.mkdir(download_path)
        
        # 対象作品のzipファイルをダウンロード
        for i, key in enumerate(zip_dict):
            print('download[{}/{}]...    {} : {}'.format(i+1, len(zip_dict), key, zip_dict[key]))
            zip_file = urllib.request.urlopen(zip_dict[key]).read()
            with open(download_path + os.path.basename(zip_dict[key]), mode="wb") as f:
                f.write(zip_file)


# ③ ダウンロードしたzipファイルを解凍
# download_zip()にてダウンロードしたzipファイルを解凍する。
def extract_zip(author_df):
    for index, row in author_df.iterrows():
        author_zip_dir = ZIP_DIR + row['author'] + os.path.sep
        author_text_sjis_dir = TEXT_SJIS_DIR + row['author'] + os.path.sep
        if not os.path.exists(author_text_sjis_dir):
            os.mkdir(author_text_sjis_dir)
 
        files = os.listdir(author_zip_dir)
        for file in files:
            root, ext = os.path.splitext(file)
            if ext == '.zip':
                with zipfile.ZipFile(author_zip_dir + file, 'r') as zf:
                    zf.extractall(path=author_text_sjis_dir)
                    print('extract {} to {}'.format(author_zip_dir + file, author_text_sjis_dir))
 
 
# ④ 解凍したテキストデータの前処理の実行
# extract_zip()にて解凍したテキストファイルに対して、不要なテキストを削除し、文字コードを変更する。
def prepare_data(author_df):
    re_ruby = re.compile('\《.+?\》')
    re_note = re.compile('\［＃.+?\］')
    for index, row in author_df.iterrows():
        author_text_sjis_dir = TEXT_SJIS_DIR + row['author'] + os.path.sep
 
        files = os.listdir(author_text_sjis_dir)
        for file in files:
            root, ext = os.path.splitext(file)
            if ext == '.txt':
 
                sjis_filename = author_text_sjis_dir + file
                fsjis = codecs.open(sjis_filename, 'r', 'shift_jis')
                blank_line_count = 0
                paragraph_index = 0
                is_header = False
                for i, line in enumerate(fsjis):
                    line = line.strip()
 
                    # 空行はスキップ
                    if len(line) == 0:
                        blank_line_count += 1
                        continue
 
                    # ヘッダはスキップ
                    if line.startswith('--------------------'):
                        is_header = not is_header
                        continue
                    if is_header:
                        continue
 
                    # フッタはスキップ
                    if 3 <= blank_line_count:
                        if line.startswith('底本：'):
                            break
                    blank_line_count = 0
 
                    # ルビ、注釈は削除
                    edited_line = re_note.sub('', re_ruby.sub('', line.replace('\r', '')))
                    if len(edited_line) == 0:
                        continue
 
                    # 段落ごとに utf8 のテキストファイルを作成
                    paragraph_index += 1
                    utf8_filename = '{}{}_{}_{}.txt'.format(TEXT_UTF8_DIR, row['author'], root, str(paragraph_index).zfill(5))
                    futf8 = codecs.open(utf8_filename, 'w', 'utf-8')
                    futf8.write(''.join(edited_line))
                    futf8.close()
 
                print('{} : {}'.format(sjis_filename, paragraph_index))
                fsjis.close()
 

# ⑤ データリストファイルの作成
# AutoMLの学習時に使用するデータリストファイルを作成する。
# データリストファイルのフォーマットは以下の２つのカラムとなる。
#   GCSのパス, 作家名のラベル
def create_data_list(filename):
    file_list = []
    label_list = []
 
    # prepare_data()にて作成された前処理後ファイルのリストを取得
    text_files = os.listdir(TEXT_UTF8_DIR)
    # GCSのパスとラベルのリストを作成
    for file in text_files:
        root, ext = os.path.splitext(file)
        file_list.append(GS_DIR + 'text/' + file)
        label_list.append(root.split('_')[0])

    # 作成したGCSのパスとラベルのリストをcsvファイルに書き込む
    data_dict = {'file_url': file_list, 'label': label_list}
    data_df = pd.DataFrame(data_dict)
    print('data_df : {}'.format(data_df))
    data_df.to_csv(ROOT_DIR + filename, index=False, header=False)


In [ ]:
# 必要なディレクトリを作成
os.makedirs(ZIP_DIR)
os.makedirs(TEXT_SJIS_DIR)
os.makedirs(TEXT_UTF8_DIR)
 
print()
print('# 青空文庫からテキストデータの zip ファイルをダウンロード ####################')
print()
download_zip(author_df)
 
print()
print('# ダウンロードした zip ファイルを解凍 ##################################')
print()
extract_zip(author_df)
 
print()
print('# 解凍したテキストデータを前処理 ####################################')
print()
prepare_data(author_df)
 
print()
print('# 学習で使用するデータリストファイルを作成 #############################')
print()
create_data_list(DATA_LIST_FILENAME)


In [ ]:
# ⑥ テキストデータファイルとデータリストファイルをGCSへアップロード
# auth.authenticate_user() でGCPへの認証を行う。
auth.authenticate_user()


In [ ]:
# テキストデータファイルをGCSへアップロードする。
# (「バケット名」は使用するバケット名に変更する)
!gsutil -m cp aozora/text_utf8/* gs://バケット名/aozora_dataset/text/
 
# データリストファイルをGCSへアップロードする。
# (「バケット名」は使用するバケット名に変更する)
!gsutil -m cp aozora/data_list.csv gs://バケット名/aozora_dataset/
